# PGM Calculation Engine Examples

These examples show how to interact with the [power-grid-model](https://github.com/PowerGridModel/power-grid-model) calculation engine to perform power flow calculations. For a detailed documentation on the calculation engine please refer to [power-grid-model docs](https://power-grid-model.readthedocs.io/en/stable/).


## Single Power Flow example

We will demonstrate how to use the PGMCoreInterface to perform Power Flow calculations on a `Grid` object.
In the examples the `RadialGridGenerator` is used to create randomised input networks.


In [1]:
from power_grid_model_ds import Grid, PowerGridModelInterface
from power_grid_model_ds.generators import RadialGridGenerator

In [ ]:
grid_generator = RadialGridGenerator(grid_class=Grid, nr_nodes=5, nr_sources=1, nr_nops=0)
grid = grid_generator.run(seed=0)

print("Created Grid:")
print(grid.branches)

core_interface = PowerGridModelInterface(grid=grid)

# Create input from grid and calculate power flow
core_interface.create_input_from_grid()
output = core_interface.calculate_power_flow()

print("Power Flow Results:")
display(output["node"])
display(output["line"])

## Updating the Grid object

If you want to store specific outputs of the calculation in the `Grid` object you can specify the columns in extended arrays. For a more detailed how to on extending arrays there is a seperate example.


In [3]:
from dataclasses import dataclass

import numpy as np
from numpy.typing import NDArray

from power_grid_model_ds import GraphContainer, Grid
from power_grid_model_ds.arrays import LineArray, NodeArray


class ExtendedNodeArray(NodeArray):
    """Extends the node array with an output value"""

    _defaults = {"u": 0}

    u: NDArray[np.float64]


class ExtendedLineArray(LineArray):
    """Extends the line array with an output value"""

    _defaults = {"i_from": 0}

    i_from: NDArray[np.float64]


@dataclass
class ExtendedGrid(Grid):
    """
    This is my own grid to extend.
    """

    node: ExtendedNodeArray
    line: ExtendedLineArray
    graphs: GraphContainer

In [ ]:
grid_generator = RadialGridGenerator(grid_class=ExtendedGrid, nr_nodes=5, nr_sources=1, nr_nops=0)
grid = grid_generator.run(seed=0)

core_interface = PowerGridModelInterface(grid=grid)
core_interface.create_input_from_grid()
core_interface.calculate_power_flow()
core_interface.update_grid()

print(grid.node)
print(grid.line)

## Batch Load Flow Example

To use the `PGM` batch calculation functionality you can supply a dictionary with update data to the `calculate_power_flow` function.


In [ ]:
from power_grid_model import initialize_array

grid_generator = RadialGridGenerator(grid_class=Grid, nr_nodes=5, nr_sources=1, nr_nops=0)
grid = grid_generator.run(seed=0)
core_interface = PowerGridModelInterface(grid=grid)

update_sym_load = initialize_array("update", "sym_load", (10, len(grid.sym_load)))
update_sym_load["id"] = [grid.sym_load.id.tolist()]
update_sym_load["p_specified"] = [grid.sym_load.p_specified.tolist()] * np.linspace(0, 1, 10).reshape(-1, 1)
update_sym_load["q_specified"] = [grid.sym_load.q_specified.tolist()] * np.linspace(0, 1, 10).reshape(-1, 1)
update_data = {
    "sym_load": update_sym_load,
}
output = core_interface.calculate_power_flow(update_data=update_data)

# Results have been calculated for all 10 scenarios
display(output["line"])

## Create grid from input data example

To create a `Grid` object from `PGM` input data, the `create_grid_from_input_data()` method can be used.

In [ ]:
input_data = core_interface.create_input_from_grid()

core_interface = PowerGridModelInterface(input_data=input_data)
output = core_interface.create_grid_from_input_data()

print(input_data["node"])
print(output.node)